# 크롤링

In [1]:
!pip install requests beautifulsoup4 -q

In [2]:
# 매장정보를 추출하는 함수(태그객체들)
page_num = 1
def get_store_info(page_num):
  url = f'https://www.hollys.co.kr/store/korea/korStore2.do?pageNo={page_num}'
  # 1. 사이트에 접속해서 html 정보 가져오기
  response = requests.get(url)
  if response.status_code == 200:
    html = response.text  
    soup = BeautifulSoup(html,'html.parser')
    # 원하는 정보를 포함하는 tag 위치 찾기
    tbody = soup.select('#contents > div.content > fieldset > fieldset > div.tableType01 > table > tbody > tr')        
    isBreak  = tbody[0].find('td').string == '등록된 지점이 없습니다.'
  else:
    print(f'error : {response.status_code}')  
  return tbody  ,isBreak

# 전달된 정보(태그객체)에서 데이터 추출
def get_store(tbody,stores):
  keys = ['resion', 'store_name', 'status', 'address', 'phone_number']
  for i in range(len(tbody)):
    values = [td.string for idx, td in enumerate(tbody[i].select('td'))  if idx != 4]
    stores.append(dict(zip(keys,values)))

In [3]:
!pip install tqdm

In [4]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

stores = []
for pagenum in tqdm(range(1,100)):  
  tbody,isBreak = get_store_info(pagenum)
  if isBreak:
    break
  get_store(tbody,stores)  

print(f'매장개수 : {len(stores)}')
print(f'페이지수 : {len(stores)//10 + 1 if len(stores)%10 else 0}')

 49%|████████████████████████████████████████▌                                         | 49/99 [00:13<00:13,  3.68it/s]

매장개수 : 483
페이지수 : 49


In [5]:
stores[:2]

[{'resion': '경남 진주시',
  'store_name': '경상국립대학생회관점',
  'status': '영업중',
  'address': '경상남도 진주시 진주대로 501 경상국립대 학생회관 1층',
  'phone_number': '055-772-0931'},
 {'resion': '서울 양천구',
  'store_name': '서서울공원점2',
  'status': '영업중',
  'address': '서울특별시 양천구 남부순환로58길 37 신월동 205-36',
  'phone_number': '070-4277-6756'}]

# 크롤링을 수집한 정보를 DB에 저장

from mysql import connector
args = {
    "host" : "localhost",
    "user" : "root",
    "password" : "root1234",    
    "port" : 3306    
}
# 접속 여부 확인
try:
    conn = connector.connect(**args)
    cursor = conn.cursor()

    # 데이터 베이스 생성
    sql = "create database IF NOT EXISTS hollycoffe";
    cursor.execute(sql)
    
    # 데이터베이스 선택
    sql = 'use hollycoffe'
    cursor.execute(sql)
    
    # 테이블 생성
    sql = '''
        create table if not exists store(
            id int AUTO_INCREMENT primary key,
            resion varchar(100)
            ,store_name varchar(100)
            ,status varchar(100)
            ,address varchar(100)
            ,phone_number varchar(100)
        )
    '''
    cursor.execute(sql)
    # 테이터 삽입
    for s in stores:        
        sql = 'insert into store values(null,%s,%s,%s,%s,%s)'        
        # bs4.element.NavigableString --> str 타입으로 변경  : varchar 속성과 일치시키기 위해서
        data = [ str(d) for d in list(s.values()) ]
        cursor.execute(sql,   data  )    
    # 모든작업이 끝나면 commit 해서 최종 반영
    conn.commit()
except Exception as e:
    print(f'error {e}')
else:
    print('정상처리 되었습니다.')
finally:  # 사용이 끝나면 자원 해제
    if cursor: 
        cursor.close()
    if conn: 
        conn.close()
          
          

https://auto.danawa.com/auto/?Work=record&Tab=Model&Nation=domestic&Month=2025-02-00

In [6]:
'''
크롤링 프로그램 순서
1. url을 통해서 사이트에 접속
2. text 정보 얻기
3. BeautiSoup 객체에 text 넣어서 soup 객체 생성
4. soup 객체를 통해 원하는 정보가 있는 태그들을 가져오기


데이터 베이스 프로그래밍 순서
1. 접속 파라메터정의
2. 접속 객체 생성
3. 접속 객체로부터 커서객체 생성

4. 쿼리 생성
5. 쿼리 실행

6. 커밋(접속객체)

7, 커서,접속 객체 close

'''

'\n크롤링 프로그램 순서\n1. url을 통해서 사이트에 접속\n2. text 정보 얻기\n3. BeautiSoup 객체에 text 넣어서 soup 객체 생성\n4. soup 객체를 통해 원하는 정보가 있는 태그들을 가져오기\n\n\n데이터 베이스 프로그래밍 순서\n1. 접속 파라메터정의\n2. 접속 객체 생성\n3. 접속 객체로부터 커서객체 생성\n\n4. 쿼리 생성\n5. 쿼리 실행\n\n6. 커밋(접속객체)\n\n7, 커서,접속 객체 close\n\n'

In [10]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

url = 'https://news.daum.net/?nil_profile=mini&nil_src=news'
response =  requests.get(url)
response.encoding = response.apparent_encoding  # 시스템에서 사용한 인코딩 자동 감지
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    li_lists = soup.select("#\\35 8d84141-b8dd-413c-9500-447b39ec29b9 > ul >li")

    news = []
    for i in tqdm(range(len(li_lists))):
      temp_dic = {}
      # print('제목 :',li_lists[i].select('strong.tit_txt')[0].text )
      temp_dic['제목'] = li_lists[i].select('strong.tit_txt')[0].text
      # if i == 0:
      #   print('내용 :',li_lists[i].select('p.desc_txt')[0].text )
      temp_dic['링크'] = li_lists[i].select('a')[0].attrs['href']
      # print('링크 :',li_lists[i].select('a')[0].attrs['href'])
      link = li_lists[i].select('a')[0].attrs['href']
      response = requests.get(link)
      response.encoding = response.apparent_encoding
      soup = BeautifulSoup(response.text, 'html.parser')
      ariticles = soup.select('#mArticle > div.news_view.fs_type1 > div.article_view > section > p')
      temp_dic['내용'] = ''.join([article.text for article in ariticles])
      news.append(temp_dic)
      # print('링크의 기사 확인 :')
      # for article in ariticles:
      #     print(article.text)

else:
    print('요청실패')



100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:01<00:00,  5.70it/s]


In [17]:
import os
import sys
import urllib.request
client_id = "sdNRbBL92SwJIst8YDmF"
client_secret = "WVBHSS_l0S"
encText = urllib.parse.quote("엔코아")
url = "https://openapi.naver.com/v1/search/news?query=" + encText # JSON 결과
# url += '&display=100'
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)

response = urllib.request.urlopen(request)
rescode = response.getcode()

import json

if(rescode==200):
    response_body = response.read()
    print(type(response_body.decode('utf-8')))
    # print(response_body.decode('utf-8'))
    json_dict = json.loads(response_body.decode('utf-8'))
    print(json_dict['items'][0]['title'])
    print(json_dict['items'][0]['link'])
    print(json_dict['items'][0]['description'])
    print(len(json_dict['items']))
else:
    print("Error Code:" + rescode)

<class 'str'>
SK네트웍스, 내년 AI 컴퍼니 전환 집중
http://www.newstomato.com/ReadNews.aspx?no=1255957&inflow=N
작년 10월 생성형 AI가 관심이던 시기 자산화 전략을 발표한 자회사 <b>엔코아</b>는 올해 신규 솔루션 패키지 출시를 준비 중입니다. 또 다른 자회사인 민팃은 AI 기술을 활용해 휴대전화 성능검사와 등급을 판별하고... 
10


In [20]:
import requests
import xml.etree.ElementTree as ET

# 🔹 공공데이터포털에서 발급받은 인증키 입력
api_key = "B%2BesHdngjG%2F9JVjFjqdEisPoXJK1b64z%2FNEquXmjvgehFIm2kiXUbcPwIOx4ox1xk53QHK6bzoBSQmkeRUHL2w%3D%3D"

# 🔹 API 요청 URL
url = f"http://apis.data.go.kr/B553077/api/open/sdsc2/storeListInDong?serviceKey={api_key}&divId=ctprvnCd&key=11&numOfRows=10&pageNo=1"

# 🔹 API 요청 보내기
response = requests.get(url)

# 🔹 응답 확인 및 XML 파싱
if response.status_code == 200:
    root = ET.fromstring(response.text)  # XML 파싱
    for item in root.findall(".//item"):
        bizesId = item.find("bizesId").text  # 상가 ID
        bizesNm = item.find("bizesNm").text  # 상호명
        brchNm = item.find("brchNm").text if item.find("brchNm") is not None else "없음"  # 지점명
        indsLclsNm = item.find("indsLclsNm").text  # 업종 대분류
        lnoAdr = item.find("lnoAdr").text  # 도로명 주소
        
        print(f"상호명: {bizesNm}, 지점명: {brchNm}, 업종: {indsLclsNm}, 주소: {lnoAdr}")

else:
    print(f"Error {response.status_code}: {response.text}")


상호명: 광명피아노, 지점명: None, 업종: 소매, 주소: 서울특별시 종로구 낙원동 288
상호명: 부동산임대김은숙, 지점명: None, 업종: 음식, 주소: 서울특별시 종로구 삼청동 28-21
상호명: 고향집, 지점명: None, 업종: 음식, 주소: 서울특별시 종로구 장사동 80
상호명: 한국황토건축연구소, 지점명: None, 업종: 부동산, 주소: 서울특별시 서초구 서초동 1308-25
상호명: 소망공인중개사사무소, 지점명: None, 업종: 부동산, 주소: 서울특별시 성동구 용답동 4-11
상호명: 보석나라, 지점명: None, 업종: 소매, 주소: 서울특별시 구로구 개봉동 359-8
상호명: 청안, 지점명: None, 업종: 소매, 주소: 서울특별시 종로구 소격동 61
상호명: 자하선인식, 지점명: None, 업종: 소매, 주소: 서울특별시 성북구 정릉동 882-6
상호명: 수피아, 지점명: None, 업종: 소매, 주소: 서울특별시 종로구 종로3가 27
상호명: 홈처치스쿨, 지점명: None, 업종: 교육, 주소: 서울특별시 동대문구 이문동 256-217


In [22]:
import requests
import json

# 🔹 공공데이터포털 API 키 입력
api_key = "B%2BesHdngjG%2F9JVjFjqdEisPoXJK1b64z%2FNEquXmjvgehFIm2kiXUbcPwIOx4ox1xk53QHK6bzoBSQmkeRUHL2w%3D%3D"

# 🔹 API URL (POST 요청)
url = "http://apis.data.go.kr/B553077/api/open/sdsc2/storeListInDong"

# 🔹 요청 헤더 (JSON 형식 명시)
headers = {
    "Content-Type": "application/json"
}

# 🔹 요청 본문 (Body)
payload = {
    "serviceKey": api_key,  # API 인증키
    "divId": "ctprvnCd",    # 조회 구분 (시도 코드)
    "key": "11",           # 서울특별시 (예시)
    "numOfRows": 10,       # 한 번에 가져올 데이터 개수
    "pageNo": 1,           # 페이지 번호
    "type": "json"         # JSON 응답 요청
}

# 🔹 API 요청 보내기 (POST 방식)
response = requests.post(url, headers=headers, data=json.dumps(payload))

# 🔹 응답 확인 및 JSON 변환
if response.status_code == 200:
    data = response.json()  # JSON 변환
    stores = data.get("body", {}).get("items", [])  # 상가 목록 가져오기

    for store in stores:
        print(json.dumps(store, ensure_ascii=False, indent=4))  # JSON 출력

else:
    print(f"Error {response.status_code}: {response.text}")


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [23]:
import requests
import json

# 🔹 공공데이터포털 API 키 입력 (URL 인코딩된 상태)
api_key = "B%2BesHdngjG%2F9JVjFjqdEisPoXJK1b64z%2FNEquXmjvgehFIm2kiXUbcPwIOx4ox1xk53QHK6bzoBSQmkeRUHL2w%3D%3D"

# 🔹 API 요청 URL (GET 방식)
url = f"http://apis.data.go.kr/B553077/api/open/sdsc2/storeListInDong?serviceKey={api_key}&divId=ctprvnCd&key=11&numOfRows=10&pageNo=1&type=json"

# 🔹 API 요청 보내기 (GET 방식)
response = requests.get(url)

# 🔹 응답 확인 및 JSON 변환
if response.status_code == 200:
    data = response.json()  # JSON 변환
    stores = data.get("body", {}).get("items", [])  # 상가 목록 가져오기
    
    # JSON 형식으로 출력
    print(json.dumps(stores, ensure_ascii=False, indent=4))  

else:
    print(f"Error {response.status_code}: {response.text}")


[
    {
        "bizesId": "MA010120220800000119",
        "bizesNm": "광명피아노",
        "brchNm": "",
        "indsLclsCd": "G2",
        "indsLclsNm": "소매",
        "indsMclsCd": "G212",
        "indsMclsNm": "기타 생활용품 소매",
        "indsSclsCd": "G21203",
        "indsSclsNm": "악기 소매업",
        "ksicCd": "G47593",
        "ksicNm": "악기 소매업",
        "ctprvnCd": "11",
        "ctprvnNm": "서울특별시",
        "signguCd": "11110",
        "signguNm": "종로구",
        "adongCd": "11110615",
        "adongNm": "종로1.2.3.4가동",
        "ldongCd": "1111013700",
        "ldongNm": "낙원동",
        "lnoCd": "1111013700102880000",
        "plotSctCd": "1",
        "plotSctNm": "대지",
        "lnoMnno": 288,
        "lnoSlno": "",
        "lnoAdr": "서울특별시 종로구 낙원동 288",
        "rdnmCd": "111102100001",
        "rdnm": "서울특별시 종로구 삼일대로",
        "bldMnno": 428,
        "bldSlno": "",
        "bldMngNo": "1111013700102540004016150",
        "bldNm": "낙원상가",
        "rdnmAdr": "서울특별시 종로구 삼일대로 428",
        "oldZ